In [1]:
!pip install ray[tune] tune-sklearn

Looking in indexes: http://pypi.douban.com/simple


You should consider upgrading via the 'd:\project\python\.venv\scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import torch
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler

data, y = torch.load('data/data_v2.pkl')
train_data, test_data = data[:len(y)], data[len(y):]

np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(train_data, y, train_size=.8, )

In [3]:
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

std = StandardScaler()
model = KNeighborsClassifier()
pipe = Pipeline(
    [('std', std),
     ('model', model)]
)

cross_val_score(pipe, X_train, y_train, cv=10, n_jobs=-1, verbose=1).mean()


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    3.0s finished


0.7921165884194052

In [7]:
# 超参数搜索
from sklearn.model_selection import GridSearchCV

params = {
    'model__n_neighbors': np.arange(1, 20),
    'model__weights': ['uniform', 'distance'],
    'model__p': [1, 2]
}
grid_search = GridSearchCV(pipe, params, cv=2, n_jobs=-1)
grid_search.fit(train_data, y)
print(grid_search.best_score_, grid_search.best_params_)



0.8226633748173527 {'model__n_neighbors': 6, 'model__p': 2, 'model__weights': 'uniform'}


In [8]:
grid_search.best_estimator_.score(train_data, y)

0.8496071829405163

In [9]:
import pandas as pd

best_model = grid_search.best_estimator_
best_model.fit(train_data, y)
y_test = best_model.predict(test_data.values)
result = pd.DataFrame(y_test, index=test_data.index, columns=['Survived'], )
result.to_csv('knn.csv')

d:\project\python\.venv\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
